# Falcon 7B Instruct Model using Perplexity Ranking

Installing Dependencies

In [1]:
# !pip install -q -U einops

Importing Dependencies

In [3]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForCausalLM

import warnings
warnings.simplefilter("ignore") # Ignore warnings

Define the model id

In [ ]:
model_id = 'tiiuae/falcon-7b-instruct'

Load tokenizer from HuggingFace library using the given model id

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token # set pad token as eos token

Load model from HuggingFace library using the given model id

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # use float16 for model for lesser memory footprint
    device_map="auto",  # automatically maps the model
    trust_remote_code=True,
    revision="main"  # use main branch of the model
)

Load the training and testing data

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Define Perplexity class to compute perplexity of model's predictions

In [ ]:
class Perplexity(nn.Module):
    def __init__(self, reduce: bool = True):
        super().__init__()
        self.loss_fn = nn.CrossEntropyLoss(reduction='mean')  # using Cross Entropy Loss
        self.reduce = reduce

    def forward(self, logits, labels):
        # Shift the labels and logits by one position to calculate loss
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()

        perplexity = self.loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        perplexity = perplexity.view(*shift_labels.size()[:-1])

        if self.reduce:
            perplexity = torch.mean(perplexity)

        return perplexity

In [ ]:
# Move Perplexity object to GPU
perplexity_object = Perplexity().to("cuda")

Function to calculate precision at k

In [ ]:
def calculate_precision_at_k(ranked_results, k):
    """
    Calculates precision at k for ranked results.
    """
    assert k <= len(ranked_results)
    assert k != 0
    return sum(int(x) for x in ranked_results[:k]) / k

Function to calculate mean average precision at 3

In [ ]:
def calculate_MAP_at_3(predictions, true_answers):
    """
    Calculates mean average precision at 3.
    """
    num_users = len(predictions)
    map_at_3_score = 0.0
    for u in range(num_users):
        user_predictions = predictions[u]
        true_answer = true_items[u]
        user_results = [1 if prediction == true_answer else 0 for prediction in user_predictions]
        for k in range(min(len(user_predictions), 3)):
            map_at_3_score += precision_at_k(user_results, k+1) * user_results[k]
    return map_at_3_score / num_users

Function to compute perplexities

In [4]:
def compute_perplexities(model, tokenizer, row):
    cols = ["A", "B", "C", "D", "E"]
    perplexities = []
    # Prepare input for the model
    samples = [f"{row['prompt']}</s>{row[col]}" for col in cols]
    inputs = tokenizer(samples, return_tensors="pt", padding=True, truncation=True).to("cuda")
    # Get model's output
    outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    # Prepare labels
    labels = inputs["input_ids"].clone()
    labels.masked_fill_(~inputs["attention_mask"].bool(), -100)
    # Calculate perplexity for each choice
    for j in range(len(cols)):
        perplexity = perp(outputs.logits[j].unsqueeze(0), labels[j].unsqueeze(0))
        perplexities.append(perplexity.detach().cpu())

    del inputs, labels, outputs, perplexity
    torch.cuda.empty_cache()  # clear GPU cache

    return np.array(perplexities)

In [ ]:
map_scores = []
all_predictions = []

For each row in the training data

In [ ]:
for _, row in tqdm(train.iterrows(), total=len(train)):
    # Compute perplexities for choices
    perplexities = compute_perplexities(model, tokenizer, row)
    cols = ["A", "B", "C", "D", "E"]
    # Sort predictions according to perplexities
    sorted_predictions = np.array(cols)[np.argsort(perplexities)]
    all_predictions.append([sorted_predictions])
    true_answer = [row.answer]
    # Calculate MAP score
    map_score = calculate_MAP_at_3([sorted_predictions], true_answer)
    map_scores.append(map_score)
    print(np.mean(map_scores))  # Print average MAP score

Mean Average Precision for the training data

In [ ]:
mean_map_score = np.mean(map_scores)
print(f"Mean Average Precision (MAP) score is: {mean_map_score}")

Load sample submission csv

In [ ]:
submission = pd.read_csv('sample_submission.csv')

Write predictions to csv file

In [ ]:
submission["prediction"] = [" ".join(p[0][:3]) for p in preds]

In [ ]:
submission.head()

Save the predictions to csv file

In [ ]:
submission.to_csv('submission.csv', index=False)